<a href="https://colab.research.google.com/github/husnain-ali21/transformers/blob/main/vanilla_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch 
import numpy as np
import torch.nn as nn
import math
import einops


In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self , heads , embedding_dim , sequence_size , batch_dim ):
        super(SelfAttention ,self).__init__() 
        self.heads = heads 
        self.embedding_dim = embedding_dim 
        self.sequence_size = sequence_size
        self.batch_dim = batch_dim

        self.transform_to_keys = nn.Linear(embedding_dim , embedding_dim * heads)
        self.transform_to_queries = nn.Linear(embedding_dim , embedding_dim*heads)
        self.transform_to_values = nn.Linear(embedding_dim , embedding_dim * heads)

        self.final = nn.Linear(heads*embedding_dim  , embedding_dim)

        def forward(self , input ,  mask = None):
            assert input.shape == (self.batch_dim ,self.sequence_size , self.embedding_dim) , "Input shape is not (b x s x e )"
            queries = einops.rearrange(self.transform_to_queries(input) , \
                                       'batch seq_size (heads emb_dim) -> (batch heads) seq_size emb_dim ' , heads = self.heads)
            keys = einops.rearrange(self.transform_to_keys(input) , \
                                   ' batch seq_size (heads emb_dim) -> (batch heads) seq_size emb_dim ' , heads = self.heads)
            values = einops.rearrange(self.transform_to_values(input) ,\
                                      'batch seq_size (heads emb_dim) -> (batch heads) seq_size emb_dim ' , heads = self.heads)
            dot_product = torch.einsum("bse,bte->bst" , [keys ,queries]) / np.sqrt(self.embedding_dim)
            if mask is not None :
                dot_product = dot_product.masked_fill(mask[None] , -np.inf)
            attention_weights = torch.nn.functional.softmax(dot_product , dim =-1)
            attention_applied_output = torch.einsum('bst,bte->bse',[attention_weights , values]) # b here actually is b*h 

            return self.final(einops.rearrange(attention_applied_output ,\
                                               "(batch heads) seq_size emb_size ->  batch seq_size (heads emb_dim)" , heads = self.heads))




In [ ]:
class TransformerBlock(nn.Module):
    def __init__(self  ,ffnn_dim , heads , embedding_dim , sequence_size , batch_dim):
        super(TransformerBlock , self).__init__() 
        self.heads = heads 
        self.embedding_dim = embedding_dim 
        self.sequence_size = sequence_size
        self.batch_dim = batch_dim

        self.norm1 = nn.LayerNorm()
        self.norm2 = nn.LayerNorm()

        self.MHA = MultiHeadAttention(self.heads , self.embedding_dim , self.sequence_size ,self.batch_dim)
        self.drop = nn.Dropout(0.3)
        self.ffnn = nn.Sequential(
            nn.Linear(embedding_dim , ffnn_dim) ,nn.LeakyReLU(0.3) , nn.Dropout(dropout) , 
            nn.Linear(ffnn ,embedding_dim) ,nn.LeakyReLU(0.3) , nn.Dropout(dropout) 
        )
        def forward(self , input , mask = None):
            output = self.norm1(self.drop(self.MHA(input , mask )) + input)
            return self.norm2(self.ffnn(output) + output)


In [ ]:
class Transformer(nn.Module):
    def __init__(self ,n_blocks , ffnn_dim , heads , embedding_dim , sequence_size , batch_dim)):
        super(Transformer , self).__init__()
        self.blocks = [TransformerBlock(ffnn_dim , heads , embedding_dim , sequence_size , batch_dim) for _ in range(n_blocks))]
        self.layers = nn.Sequential(self.blocks)

        def forward(self , input , mask) = None:
            return self.layers(input)

            
